<a href="https://colab.research.google.com/github/victoriawermelinger/SoulCode/blob/main/colabbdor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Criar tipos compostos (endereço e telefone)

In [ ]:
-- Criar tipo para endereço
CREATE TYPE tp_endereco AS (
    rua VARCHAR(100),
    numero INT,
    bairro VARCHAR(50),
    cidade VARCHAR(50),
    cep CHAR(8)
);

-- Criar tipo para telefone
CREATE TYPE tp_telefone AS (
    ddd CHAR(3),
    numero CHAR(9)
);

##Criar a tabela PESSOA (tabela "pai")

In [ ]:
CREATE TABLE IF NOT EXISTS PESSOA (
    id SERIAL PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    endereco tp_endereco,  -- ROW contendo o tipo composto endereco
    telefones tp_telefone[]  -- ARRAY de tipo telefone
);

##Criar a tabela PESSOA_FISICA (herança de PESSOA)

In [ ]:
CREATE TABLE PESSOA_FISICA (
    cpf CHAR(11) UNIQUE NOT NULL,  -- Atributo específico de Pessoa Física
    data_nascimento DATE
) INHERITS (PESSOA);


##Criar a tabela PESSOA_JURIDICA (herança de PESSOA)

In [ ]:
CREATE TABLE PESSOA_JURIDICA (
    cnpj CHAR(14) UNIQUE NOT NULL,  -- Atributo específico de Pessoa Jurídica
    razao_social VARCHAR(150) NOT NULL,
    nome_fantasia VARCHAR(150)
) INHERITS (PESSOA);


##Inserindo Pessoa_Fisica

In [ ]:
#Com apenas um telefone
INSERT INTO PESSOA_FISICA (nome, endereco, telefones, cpf, data_nascimento) VALUES
('João Silva',('Rua das Flores', 123, 'Centro', 'São Paulo', '01000-000'),
 ARRAY[ROW('011', '987654321')::tp_telefone],  -- Especificar explicitamente o tipo telefone
 '12345678901', '1980-05-12');

#Com mais de um telefone
INSERT INTO PESSOA_FISICA (nome, endereco, telefones, cpf, data_nascimento) VALUES
('Maria Oliveira',
 ('Avenida Brasil', 456, 'Jardim América', 'Rio de Janeiro', '22000-000'),
 ARRAY[ROW('021', '987654322')::tp_telefone, ROW('021', '912345678')::tp_telefone],  -- Inserir múltiplos telefones
 '98765432100', '1990-08-25');


In [ ]:
#Consulta Pessoa Fisica
SELECT nome, data_nascimento,(endereco).rua, (endereco).numero, (endereco).bairro, (endereco).cidade, (endereco).cep FROM pessoa_fisica

In [ ]:
#Criando View
CREATE VIEW CONSULTA_PF AS
SELECT nome, data_nascimento,(endereco).rua, (endereco).numero, (endereco).bairro, (endereco).cidade, (endereco).cep FROM pessoa_fisica

In [ ]:
#Inserindo Pessoa Juridica
INSERT INTO PESSOA_JURIDICA (nome, endereco, telefones, cnpj, razao_social, nome_fantasia) VALUES
('Empresa XYZ',
    ('Avenida Brasil', 456, 'Centro', 'Rio de Janeiro', '22000-000'),
    ARRAY[ROW('021', '987654322')::tp_telefone],  -- Especificar explicitamente o tipo telefone
    '12345678000195', 'XYZ S.A.', 'XYZ');


##BDOR Sem usar ARRAY

In [ ]:
-- Criar tipo para endereço
CREATE TYPE tp_endereco AS (
    rua VARCHAR(100),
    numero INT,
    bairro VARCHAR(50),
    cidade VARCHAR(50),
    cep CHAR(8)
);

-- Criar tipo para telefone
CREATE TYPE tp_telefone AS (
    ddd CHAR(3),
    numero CHAR(9)
);

CREATE TABLE IF NOT EXISTS PESSOA (
    id SERIAL PRIMARY KEY,
    nome VARCHAR(100) NOT NULL,
    endereco tp_endereco,  -- ROW contendo o tipo composto endereco
    telefones tp_telefone  -- ROW de tipo telefone
);

CREATE TABLE PESSOA_FISICA (
    cpf CHAR(11) UNIQUE NOT NULL,  -- Atributo específico de Pessoa Física
    data_nascimento DATE
) INHERITS (PESSOA);

CREATE TABLE PESSOA_JURIDICA (
    cnpj CHAR(14) UNIQUE NOT NULL,  -- Atributo específico de Pessoa Jurídica
    razao_social VARCHAR(150) NOT NULL,
    nome_fantasia VARCHAR(150)
) INHERITS (PESSOA);





##Inserção de Pessoa Física

In [ ]:
INSERT INTO PESSOA_FISICA (nome, endereco, telefones, cpf, data_nascimento)
VALUES (
    'João da Silva',
    ROW('Rua A', 123, 'Centro', 'São Paulo', '12345678'),
    ROW('11', '987654321'),
    '12345678901',
    '1980-01-01'
);


##Inserção de Pessoa Jurídica

In [ ]:
INSERT INTO PESSOA_JURIDICA (nome, endereco, telefones, cnpj, razao_social, nome_fantasia)
VALUES (
    'Empresa XYZ Ltda',
    ROW('Avenida B', 456, 'Jardim', 'São Paulo', '87654321'),
    ROW('11', '912345678'),
    '12345678000195',
    'Empresa XYZ',
    'XYZ'
);


In [ ]:
CREATE VIEW DADOS_PESSOA_FISICA AS
SELECT nome, cpf, data_nascimento, (endereco).rua, (endereco).numero, (endereco).bairro, (endereco).cidade, (endereco).cep, (telefones).ddd, (telefones).numero as telefone FROM PESSOA_FISICA

##Criando tabelas usando OF

In [ ]:
CREATE TYPE tipo_endereco AS (
    rua VARCHAR(100),
    numero INT,
    bairro VARCHAR(50),
    cidade VARCHAR(50),
    cep CHAR(8)
);

CREATE TYPE tipo_dados as (
  nome VARCHAR(100),
  telefone VARCHAR(20),
  cpf VARCHAR(14),
  endereco_completo tipo_endereco,
  email VARCHAR(100)
)


CREATE TABLE CLIENTE OF tipo_dados;
CREATE TABLE ESTUDANTE OF tipo_dados;

select * from CLIENTE;



CREATE TABLE alunos (
    matricula SERIAL PRIMARY KEY,
    dados tipo_dados
);



INSERT INTO alunos (dados) VALUES (
    ROW(
        'João Silva',              -- nome
        '(11) 98765-4321',         -- telefone
        '123.456.789-00',          -- cpf
        ROW(
            'Rua das Flores',     -- rua
            123,                  -- numero
            'Centro',             -- bairro
            'São Paulo',          -- cidade
            '12345678'            -- cep
        )::tipo_endereco,
        'joao.silva@example.com'  -- email
    )::tipo_dados
);






select * from alunos;

CREATE VIEW CONSULTA_ALUNOS AS
select matricula, (dados).nome, (dados).cpf, (dados).telefone, (dados).endereco_completo.rua, (dados).endereco_completo.numero, (dados).endereco_completo.bairro, (dados).endereco_completo.cidade, (dados).endereco_completo.cep, (dados).email from alunos;
